# Ray Tune - A Deeper Dive Using MNIST with PyTorch

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

A [previous notebook](02-Understanding-Hyperparameter-Tuning.ipynb) explained the concept of hyperparameter tuning/optimization (HPO) and walked through the basics of using [Ray Tune](https://ray.readthedocs.io/en/latest/tune.html), and another [notebook on Tune and Sklearn](03-Ray-Tune-with-Sklearn.ipynb) showed Tune's drop-in replacements for HPO.

Now we'll use another example to explore more of the Tune API features. We'll use the [MNIST](http://yann.lecun.com/exdb/mnist/) of hand-written digits and train a [PyTorch](https://pytorch.org/) model to recognize them.

In [1]:
import os 
from torchvision import datasets, transforms
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from filelock import FileLock

## PyTorch Hyperparameter Tuning

Our example will closely follow the code in the [PyTorch MNIST example](https://github.com/pytorch/examples/blob/master/mnist/main.py). However, we will create an even simpler model than the one in the example, although you could try that model and compare its predictions.

Let's start by defining a few global variables for epoch and test sizes. Also define a data location.

In [2]:
EPOCH_SIZE = 512
TEST_SIZE = 256

DATA_ROOT = '../data/mnist'

The following class defines a convolutional neural network.

> **Tip:** Most of these code definitions can be found in `mnist.py`, too.

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

After creating that network, we can now create our data loaders for training and test data. These are just plain [PyTorch `DataLoaders`](https://pytorch.org/docs/1.1.0/data.html?highlight=dataloader#torch.utils.data.DataLoader) with two additions:

1. A `FileLock` is added to ensure that only one process downloads the data on each machine, just in case we have multiple workers per machine in our Ray cluster.
2. The root directory for the data can be specified and it will be created if it doesn't exist.

Otherwise, this code is identical to the [PyTorch example version](https://github.com/pytorch/examples/blob/master/mnist/main.py#L101).

In [4]:
def get_data_loaders():
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    # We add FileLock here because multiple workers on the same machine coulde try 
    # download the data. This would cause overwrites, since DataLoader is not threadsafe.
    # You wouldn't need this for single-process training.
    lock_file = f'{DATA_ROOT}/data.lock'
    import os
    if not os.path.exists(DATA_ROOT):
        os.makedirs(DATA_ROOT)
        
    with FileLock(os.path.expanduser(lock_file)):
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST(DATA_ROOT, train=True, download=True, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)

        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST(DATA_ROOT, train=False, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)
    return train_loader, test_loader

Now we define our training and test functions. While the arguments are a bit switched up from the original PyTorch tutorial, the difference is inconsequential. The arguments are an optimizer, a model, the training data loader, and our device. Then we train the model.

In [5]:
def train(model, optimizer, train_loader, device=torch.device("cpu")):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

Similarly for our test model, we define a basic _average correct prediction_ metric that we will track. We could add more metrics, but we'll keep it simple.

In [6]:
def test(model, data_loader, device=torch.device("cpu")):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total

Finally, we create a wrapper function for this particular model. In doing so all we need to do is specify the configuration for the model that we would like to train and the function will do the rest:

1. Retrieve the data with the loaders returned by `get_data_loaders()`
2. Create the `ConvNet` model
3. Optimize the model using _stochastic gradient descent_.

In [7]:
def train_mnist(config):
    train_loader, test_loader = get_data_loaders()
    model = ConvNet()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config['momentum'])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)
        print(f"accuracy: {acc}")

### Single-Node Hyperparameter Tuning

Let's show what we might do if we performed hyperparameter tuning on a single machine. We would have to enumerate all the possibilities and either train them serially or use something like multiprocessing to train them in parallel. That setup takes a little bit of work so people often decide to train them serially, which is easiest, but requires the most time.

This is what we might do.

In [8]:
import itertools
conf = {
    "lr": [0.001, 0.01, 0.1],
    "momentum": [0.001, 0.01, 0.1, 0.9]
}

combinations = list(itertools.product(*conf.values()))
print(len(combinations))
combinations

12


[(0.001, 0.001),
 (0.001, 0.01),
 (0.001, 0.1),
 (0.001, 0.9),
 (0.01, 0.001),
 (0.01, 0.01),
 (0.01, 0.1),
 (0.01, 0.9),
 (0.1, 0.001),
 (0.1, 0.01),
 (0.1, 0.1),
 (0.1, 0.9)]

In [9]:
for lr, momentum in combinations:
    train_mnist({"lr":lr, "momentum":momentum})
    break # we'll stop this after one run and just use it for illustrative purposes

/usr/local/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1635217250457/work/torch/csrc/utils/tensor_numpy.cpp:189.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


accuracy: 0.125
accuracy: 0.15625
accuracy: 0.153125
accuracy: 0.140625
accuracy: 0.159375
accuracy: 0.165625
accuracy: 0.16875
accuracy: 0.19375
accuracy: 0.16875
accuracy: 0.196875


### Distributed Hyperparameter Tuning with Ray Tune

Ray Tune makes it trivial to move this code from a single node to multiple nodes. Let's see how to use the code we've written with Ray Tune.

First, we set up Ray as before.

In [10]:
import ray
from ray import tune

In [11]:
ray.init(ignore_reinit_error=True)

2022-02-21 11:53:15,024	INFO services.py:1376 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-02-21_11-53-12_309410_14604/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-21_11-53-12_309410_14604/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-02-21_11-53-12_309410_14604',
 'metrics_export_port': 60000,
 'gcs_address': '127.0.0.1:55789',
 'node_id': 'addf869252d523ad15f5c44be013e55f53c5b8efe9028d62586613bb'}

The first change is we'll perform a strict `grid_search` on our hyperparameters, like we used in the previous lesson. Our hyperparameters are the learning rate, `lr`, and the `momentum`.

In [12]:
config = {
    "lr": tune.grid_search([0.001, 0.01, 0.1]),
    "momentum": tune.grid_search([0.001, 0.01, 0.1, 0.9])
}

Next we modify our trainable, `train_mnist`, to use Tune's "reporting" logger:

In [13]:
def train_mnist(config):
    from ray.tune import report
    train_loader, test_loader = get_data_loaders()
    model = ConvNet()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config['momentum'])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)
        # This sends the score to Tune.
        report(mean_accuracy=acc)

That's all that we need to change in order for Ray Tune to be able to parallelize our different hyperparameter combinations. 

When we execute a hyperparameter sweep, we perform an **experiment**. Each distinct combination of our different hyperparameters constitutes a single **trial**.

## Tune's Functional vs. Class API

In the above previous lesson, we used the **functional API**. This API is most convenient for quickly setting up experiments, but it provides less overall flexbility compared to the **class API** [`tune.Trainable`](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#tune-trainable).

We'll try both, starting with the functional API.

We add a stopping criterion, `stop={"training_iteration": 20}`, so this will go reasonably quickly, while still producing good results. Consider removing this condition if you don't mind waiting longer and you want optimal results.

**Note**: Unlike the functional API, in which you the trainable can call a `tune.report()`, the class API method `cls.step()` can only return a value.

In [14]:
%%time
analysis_func = tune.run(train_mnist, config=config, stop={"training_iteration": 20},
                         verbose=1)

2022-02-21 11:54:06,562	INFO tune.py:636 -- Total run time: 7.61 seconds (7.27 seconds for the tuning loop).


CPU times: user 988 ms, sys: 255 ms, total: 1.24 s
Wall time: 7.65 s


In [15]:
print("Best config: ", analysis_func.get_best_config(metric="mean_accuracy", mode="max"))

Best config:  {'lr': 0.1, 'momentum': 0.1}


In [16]:
analysis_func.dataframe().sort_values('mean_accuracy', ascending=False).head()

,mean_accuracy,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/lr,config/momentum,logdir
8,0.896875,0.304352,False,NaN,NaN,10,02766_00008,cb239a5d5a4b455792050480a658ba1d,2022-02-21_11-54-06,1645473246,3.588721,14724,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,3.588721,0,10,0.10,0.100,/Users/jules/ray_results/train_mnist_2022-02-2...
2,0.884375,0.303644,False,NaN,NaN,10,02766_00002,adea9b269fba42e28f0ac624e4602eaf,2022-02-21_11-54-05,1645473245,3.599486,14732,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,3.599486,0,10,0.10,0.001,/Users/jules/ray_results/train_mnist_2022-02-2...
5,0.884375,0.287993,False,NaN,NaN,10,02766_00005,3bc58cca1bc44936916d482628bf86b4,2022-02-21_11-54-06,1645473246,3.760321,14728,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,3.760321,0,10,0.10,0.010,/Users/jules/ray_results/train_mnist_2022-02-2...
10,0.878125,0.242178,False,NaN,NaN,10,02766_00010,431bc22437f040a784b0df3c9f35b280,2022-02-21_11-54-06,1645473246,3.832313,14722,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,3.832313,0,10,0.01,0.900,/Users/jules/ray_results/train_mnist_2022-02-2...
11,0.856250,0.281464,False,NaN,NaN,10,02766_00011,4f10c07120a748a1ab47ab6af364b033,2022-02-21_11-54-06,1645473246,3.812603,14721,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,3.812603,0,10,0.10,0.900,/Users/jules/ray_results/train_mnist_2022-02-2...


In [17]:
analysis_func.dataframe()[['mean_accuracy', 'config/lr', 'config/momentum']].sort_values('mean_accuracy', ascending=False)

,mean_accuracy,config/lr,config/momentum
8,0.896875,0.100,0.100
2,0.884375,0.100,0.001
5,0.884375,0.100,0.010
10,0.878125,0.010,0.900
11,0.856250,0.100,0.900
7,0.784375,0.010,0.100
1,0.725000,0.010,0.001
9,0.450000,0.001,0.900
4,0.343750,0.010,0.010
0,0.134375,0.001,0.001


How long did it take? We'll compare this value with a different training run in the next lesson.

In [18]:
stats = analysis_func.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

   0.20 seconds,    0.00 minutes


### Use Tune's Trainable Class API

As a subclass of `tune.Trainable`, Tune will create a Trainable object on a separate process (using the [Ray Actor API](https://docs.ray.io/en/latest/actors.html#actor-guide)).

 * setup function is invoked once training starts.
 * step is invoked multiple times. Each time, the Trainable object executes one logical iteration of training in the tuning process, which may include one or more iterations of actual training.


In [19]:
class TrainMNIST(tune.Trainable):
    def setup(self, config):
        self.config = config
        self.train_loader, self.test_loader = get_data_loaders()
        self.model = ConvNet()
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.config["lr"])
    
    def step(self):
        train(self.model, self.optimizer, self.train_loader)
        acc = test(self.model, self.test_loader)
        return {"mean_accuracy": acc}

In [20]:
%%time
analysis = tune.run(
    TrainMNIST, 
    config=config,
    stop={"training_iteration": 20},
    verbose=1
)

2022-02-21 11:55:01,905	INFO tune.py:636 -- Total run time: 12.60 seconds (12.45 seconds for the tuning loop).


CPU times: user 2.37 s, sys: 468 ms, total: 2.84 s
Wall time: 12.6 s


In [21]:
print("Best config: ", analysis.get_best_config(metric="mean_accuracy", mode="max"))

Best config:  {'lr': 0.1, 'momentum': 0.001}


In [22]:
# Get a dataframe for analyzing trial results.
df = analysis.dataframe()
df.head()

,mean_accuracy,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/lr,config/momentum,logdir
0,0.225000,True,NaN,NaN,20,2079a_00000,d4e07b4ab61540258ce7137f6b22ee8e,2022-02-21_11-55-01,1645473301,0.318386,6.481498,14926,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.481498,0,20,0.001,0.001,/Users/jules/ray_results/TrainMNIST_2022-02-21...
1,0.846875,True,NaN,NaN,20,2079a_00001,1de6220bb4374a3eafe7943317b7096d,2022-02-21_11-55-01,1645473301,0.291061,6.543176,14924,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.543176,0,20,0.010,0.001,/Users/jules/ray_results/TrainMNIST_2022-02-21...
2,0.921875,True,NaN,NaN,20,2079a_00002,b99e423fed144e43a63ae4cb2f3e65a1,2022-02-21_11-55-01,1645473301,0.285952,6.454046,14917,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.454046,0,20,0.100,0.001,/Users/jules/ray_results/TrainMNIST_2022-02-21...
3,0.190625,True,NaN,NaN,20,2079a_00003,8924290b2d35403b8cae7df5814c6d45,2022-02-21_11-55-01,1645473301,0.323717,6.314408,14921,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.314408,0,20,0.001,0.010,/Users/jules/ray_results/TrainMNIST_2022-02-21...
4,0.678125,True,NaN,NaN,20,2079a_00004,3f6ead58137c4eeab12f52d56ba11cb4,2022-02-21_11-55-01,1645473301,0.309406,6.444861,14919,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.444861,0,20,0.010,0.010,/Users/jules/ray_results/TrainMNIST_2022-02-21...


In [23]:
analysis.dataframe().sort_values('mean_accuracy', ascending=False).head()

,mean_accuracy,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/lr,config/momentum,logdir
2,0.921875,True,NaN,NaN,20,2079a_00002,b99e423fed144e43a63ae4cb2f3e65a1,2022-02-21_11-55-01,1645473301,0.285952,6.454046,14917,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.454046,0,20,0.10,0.001,/Users/jules/ray_results/TrainMNIST_2022-02-21...
11,0.912500,True,NaN,NaN,20,2079a_00011,15cb1e80f33c42b3b3ed4d81cd3b7aac,2022-02-21_11-55-01,1645473301,0.301283,6.367992,14952,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.367992,0,20,0.10,0.900,/Users/jules/ray_results/TrainMNIST_2022-02-21...
5,0.906250,True,NaN,NaN,20,2079a_00005,23faa290f2754b759924fcbafdb642de,2022-02-21_11-55-01,1645473301,0.312569,6.361443,14923,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.361443,0,20,0.10,0.010,/Users/jules/ray_results/TrainMNIST_2022-02-21...
7,0.884375,True,NaN,NaN,20,2079a_00007,5b012323c8104b9398284de86fb8b83c,2022-02-21_11-55-01,1645473301,0.321634,6.381609,14922,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.381609,0,20,0.01,0.100,/Users/jules/ray_results/TrainMNIST_2022-02-21...
8,0.878125,True,NaN,NaN,20,2079a_00008,e00d07d9117d48dfa3aba76a1480f72c,2022-02-21_11-55-01,1645473301,0.253248,6.430459,14916,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.430459,0,20,0.10,0.100,/Users/jules/ray_results/TrainMNIST_2022-02-21...


It's easier to see what we want if project out the interesting columns:

In [24]:
analysis.dataframe()[['mean_accuracy', 'config/lr', 'config/momentum']].sort_values('mean_accuracy', ascending=False)

,mean_accuracy,config/lr,config/momentum
2,0.921875,0.100,0.001
11,0.912500,0.100,0.900
5,0.906250,0.100,0.010
7,0.884375,0.010,0.100
8,0.878125,0.100,0.100
1,0.846875,0.010,0.001
10,0.800000,0.010,0.900
4,0.678125,0.010,0.010
6,0.271875,0.001,0.100
0,0.225000,0.001,0.001


How long did it take? We'll compare this value with a different training run in the next lesson.

In [25]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

  10.24 seconds,    0.17 minutes


The next lesson will explore optimization algorithms that speed up HPO.

In [26]:
ray.shutdown()  # "Undo ray.init()".